In [ ]:
THIS IS THE SIMPLEST TIME INDEXED DATAFRAME THAT I CAN CONCEIVE.
IT IS INDEXED ON AN INTEGER THAT CAN REPRESENT A UNIX TIME STAMP.
THIS IS A FUNCTIONING COMPONENT TO INCLUDE IN NEW PARADIGM.

In [8]:
def cleanenv():
    global testmode
    if 'testmode' in globals() or 'testmode' in locals():
        del testmode
        print('testmode deleted')
    else:
        print('testmode does not exist')
    global df0
    if 'df0' in globals() or 'df0' in locals():
        del df0
        print('df0 deleted')
    else:
        print('df0 does not exist')           

In [9]:
# --------- MAKEIBDF() ---------- FILENAME() ---------- SLEEPTIME() ----------
def makeibdf(x,y):
    df0 = pd.DataFrame({'dty' : [x]},index=y)
    df0['bid']       =0.0   # 1
    df0['rate']      =0.0   # 2 
    df0['ask']       =0.0   # 3
    df0['refprice']  =0.0   # 4
    df0['chapc']     =0.0   # 5M NEWCUMFUN
    df0['trigger1']  =0.0   # 6M
    df0['trigger2']  =0.0   # 7M
    df0['comm']      =0.0   # 8M
    df0['unitsii']   =0     # 9M
    df0['idlh']      =''    # 10M
    df0['fope']      =0     # 11M
    df0['repct']     =0     # 12M
    df0['revct']     =0     # 13M
    df0['winsinstrk']=0     # 14M
    df0['strknumb']  =0     # 15M
    dfx=df0[['dty','bid','rate','ask','refprice','chapc','trigger1','trigger2','comm','unitsii','idlh','fope','repct','revct','winsinstrk','strknumb']]
    return dfx
def filename():
    global testmode
    fmt      = '%Y-%m-%d'
    d        = datetime.now(pytz.timezone('UTC'))
    d_string = d.strftime(fmt)
    if testmode:
        return 'ib_test_'+ d_string.replace('-','_').replace(':','_').replace(' ','_')
    else:
        return 'ib'+ d_string.replace('-','_').replace(':','_').replace(' ','_')
def sleeptime():
    t = datetime.utcnow()
    timetosleep = 60 - (t.second + t.microsecond/1000000.0)
    return timetosleep 
def usebidorask(bid,ask,nunitsii):  # NEEDS CODE FOR DD.
    if nunitsii > 0:
        price=bid
    else:
        price=ask
    return price

In [10]:
# ---------- TRIGGERF ---------- CUMFUN ---------- FUN1 ----------
def triggerf(tf1,tf2,unitsii):  
    if abs(unitsii)   ==0:
        tf=tf1
    elif abs(unitsii) ==1:
        tf=tf1
    elif abs(unitsii) ==3:
        tf=tf2
    return tf
def cumfun(price,refprice):
    return round((np.log(price) - np.log(refprice)),7)
def fun1(price,refprice,ounitsii,trigger): 
    trtp='tradetype'
    if   price>refprice and ounitsii== 0: #up and were out   up0
        nunitsii,trtp,wld =-1,  'S1TEI', 'd'
    elif price<refprice and ounitsii== 0: #dn and were out   dn0
        nunitsii,trtp,wld = 1 , 'B1TEI', 'd'
    elif price>refprice and ounitsii== 1: #up and were long  up1
        nunitsii,trtp,wld =-1,  'S2TLP', 'w'
    elif price>refprice and ounitsii==-1: #up and were short up-1
        nunitsii,trtp,wld =-3,  'S2TDD', 'd'
    elif price>refprice and ounitsii== 3: #up and were long  up3
        nunitsii,trtp,wld =-1,  'S4TLP', 'w'
    elif price>refprice and ounitsii==-3: #up and were short up-3
        nunitsii,trtp,wld = 1,  'B4TLL', 'l'
    elif price<refprice and ounitsii== 1: #up and were long  up1
        nunitsii,trtp,wld = 3,  'B2TDD', 'd'
    elif price<refprice and ounitsii==-1: #up and were short up-1
        nunitsii,trtp,wld = 1,  'B2TLP', 'w'
    elif price<refprice and ounitsii== 3: #up and were long  up3
        nunitsii,trtp,wld  =-1, 'S4TLL', 'l'
    elif price<refprice and ounitsii==-3: #up and were short up-3
        nunitsii,trtp,wld  = 1, 'B4TLP', 'w'
    return nunitsii, trtp, wld

In [59]:
# ---------- IBPY INTERFACE -- FETCHBAT() ----------
# THIS MODULE CONTAINS ALL OF THE CODE REQUIRED TO COMMUNICATE WITH TWS.
# THIS MODULE WILL GET BID, ASK AND TIME FROM TWS AND PUT THEM IN GLOBAL VARIABLES.
# THE FUNCTION FETCHBAT() (bid,ask,time) DOES THE WORK. CALL IT EACH TIME YOU WANT UPDATED QUOTE.
from ib.opt import Connection, message
from ib.ext.Contract import Contract
from datetime import datetime
import pytz, tzlocal
from time import sleep
global bid,ask,excthangetime,testmode
def raise_error(msg):
    assert(msg.typeName == 'error')
    raise Exception("We've received an error message from IB: {}".format(msg))
def extract_time_from_currentTime_message(msg):
    global xtime
    xtime         = msg.time
    nytime        = datetime.fromtimestamp(msg.time, pytz.timezone('US/Eastern'))
    print(nytime)
    localtime     = datetime.fromtimestamp(msg.time, tzlocal.get_localzone())
    utctime       = datetime.fromtimestamp(msg.time, pytz.timezone('UTC'))
    minutetime    = round((utctime.hour*60+utctime.minute+utctime.second/60),2)
def makeStkContract(sym):
    newStkContract = Contract()
    newStkContract.m_symbol = sym
    newStkContract.m_secType = 'CASH'
    newStkContract.m_strike = 0.00
    newStkContract.m_exchange = 'IDEALPRO'
    newStkContract.m_currency = 'USD'
    return newStkContract
    #sym,'CASH',0.00,'IDEALPRO','USD'
def print_message_from_ib(msg):
    global bid,ask
    if msg.field==1:
        bid=(msg.price)
    if msg.field==2:
        ask=(msg.price)
# ---------- FETCHBAT() ----------
def fetchbat(i): 
    if testmode: # USE i=0 INSTEAD OF TESTMODE?
        if 'newfeed' not in locals():
            newfeed      = pd.read_csv('c:\\thomas\\ib\\2016_10_21_5sec_a_.csv',index_col=0)
            newfeed.index= pd.to_datetime(newfeed.index)
            newfeed.index= newfeed.index.tz_localize(pytz.utc)
        return newfeed.iloc[i,0],newfeed.iloc[i,1],newfeed.index[i]    
    else:
        global bid,ask,xtime
        conn = Connection.create(port=7497, clientId=1329)
        conn.registerAll(print_message_from_ib)
        conn.register(extract_time_from_currentTime_message, 'CurrentTime')
        conn.register(raise_error, 'Error')
        #conn.register(print_message_from_ib, 'msg.tickPrice')
        conn.connect()
        tickId = 24
        stkContract = makeStkContract('EUR')
        conn.reqMktData(tickId, stkContract, '',0)
        conn.reqCurrentTime()
        import time
        time.sleep(1) #Simply to give the program time to print messages sent from IB
        conn.disconnect() 
        return bid,ask,xtime

In [12]:
import os, sys
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
import pickle
import aTcmodule
import aTcmodule as tcm
from IPython.display import clear_output
import numexpr 
import bottleneck
import winsound
import pytz
import time
import datetime as dt
import datetime as datetime
from datetime import datetime, timedelta
from pytz import timezone

In [ ]:
# ---------- LIMITS OF CONTROL ----------
global testmode
testmode =False
file     =filename()                                            # FILENAME CHANGES FOR TESTMODE
tf1, tf2 = 0.00020, 0.00020
profitloss, comm = 0, 0
winsinstrk, strknumb,  repct,  revct, trtp =0, 0, 0, 0, ''
print              ('TestMode: '+str(testmode));
if 'df0' not in globals() or 'df0' not in locals():
    try:
        print      ('df0 NOT found in memory.');
        print      ('Looking for '+file+' on disk.');
        df0=pd.read_csv('c:\\thomas\\ib\\'+file+'.csv', index_col=0)
    except:
        print       ('df0 not found on disk, makenig new df0');
        bid,ask,xtime=fetchbat(0)
        df0          =makeibdf('',[xtime])
    df0.iloc[0,1], df0.iloc[0,2], df0.iloc[0,3], df0.iloc[0,4] = bid, (bid+ask)/2, ask, (bid+ask)/2
    df0.iloc[0,0]    =datetime.fromtimestamp(xtime, pytz.timezone('UTC')).hour*60+datetime.fromtimestamp(xtime, pytz.timezone('UTC')).minute
else:
    print            ('df0 found in memory.');
for i in range (3333):
    time.sleep(4)
    clear_output()
    start = dt.datetime.now()
    clear_output()
    print            ('update dfx and append to df0'); 
    bid,ask,xtime =fetchbat(0)
    price         =usebidorask(bid,ask,df0.iloc[df0.shape[0]-1,9])
    dfx           =makeibdf('',[xtime])
    dfx.iloc[0,0] =datetime.fromtimestamp(xtime, pytz.timezone('UTC')).hour*60+datetime.fromtimestamp(xtime, pytz.timezone('UTC')).minute
    dfx.iloc[0,1], dfx.iloc[0,2], dfx.iloc[0,3] = bid, price, ask
    ncumfun       =cumfun     (price,  df0.iloc[df0.shape[0]-1,4])
    trigger       =triggerf   (tf1,tf2,df0.iloc[df0.shape[0]-1,9])
    if abs(ncumfun)>trigger:                                                  # REF PRICE AND UNITSII CHANGE.
        nunitsii,trtp,wld  = fun1(price,df0.iloc[df0.shape[0]-1,4],df0.iloc[df0.shape[0]-1,9],trigger)
        dfx.iloc[0,4]      = price
        dfx.iloc[0,5]      = ncumfun
        dfx.iloc[0,6]      = trigger
        dfx.iloc[0,8]      = ((abs(df0.iloc[df0.shape[0]-1,9])+1)*2)           # COMM
        dfx.iloc[0,9]      = nunitsii
        dfx.iloc[0,10]     = trtp
        dfx.iloc[0,11]     = (ncumfun*df0.iloc[df0.shape[0]-1,9]*price*100000) # FOPE
        timeoftrade        = dfx.iloc[0,0]
        if ncumfun*df0.iloc[df0.shape[0]-1,9]> 0:                              # REVERSAL
            repct +=1
            dfx.iloc[0,12] =revct
            dfx.iloc[0,13] =repct
        else:
            revct +=1
            dfx.iloc[0,12] =revct
            dfx.iloc[0,13] =repct
        if wld=='w':                                                             # WINNING STREAK
            winsinstrk +=1
            dfx.iloc[0,14] =winsinstrk
        elif wld=='l':
            dfx.iloc[0,14] =winsinstrk
            winsinstrk =0
            strknumb +=1
            dfx.iloc[0,15] =strknumb
        else:
            pass
    else: # NO TRANSACTION, ONLY PRICE CHANGES, UNITSII CARRYED FORWARD.
        dfx.iloc[0,4]     = df0.iloc[df0.shape[0]-1,4]
        dfx.iloc[0,5]     = ncumfun
        dfx.iloc[0,9]     = df0.iloc[df0.shape[0]-1,9]
    df0                   = df0.append(dfx)
    #clear_output()
    print('............................................................................... ') 
    print(datetime.fromtimestamp(xtime, pytz.timezone('UTC')))
    print('Iteration. . . . . . . . . . . . . . . . . . . :' + str(i))
    print('Records. . . . . . . . . . . . . . . . . . . . :' + str(df0.shape[0]))
    print('Trades. . . . . . . . . . . . . . . . . . . . .:' + str(df0['idlh'].count()))
    print('Last Trade Type. . . . . . . . . . . . . . . . :' + trtp)
    print('Minutes. . . . . . . . . . . . . . . . . . . . :' + str(df0.iloc[df0.shape[0]-1,0]))
    print('Time of Trade. . . . . . . . . . . . . . . . . :' + str(timeoftrade))
    print('Reference Price. . . . . . . . . . . . . . . . :' + str(df0.iloc[df0.shape[0]-1,4]))
    print('Price. . . . . . . . . . . . . . . . . . . . . :' + str(price))
    print('Trigger. . . . . . . . . . . . . . . . . . . . :' + str("%.5f" % trigger)) 
    print('Limit of Control . . . . . . . . . . . . . . . :' + str("%.5f" % (trigger-abs(ncumfun))))
    print('Cumulative Profit/Loss. . . . . .  . . . . . . :' + str(int(df0['fope'].sum())))
    print('Cumulative Commissions . . . . . . . . . . . . :' + str(df0['comm'].sum()))
    print('Wins in Streak . . . . . . . . . . . . . . . . :' + str(winsinstrk)) 
    print('Streak Number. . . . . . . . . . . . . . . . . :' + str(strknumb))
    print('Reversal Count . . . . . . . . . . . . . . . . :' + str(revct))
    print('Repeat Count . . . . . . . . . . . . . . . . . :' + str(repct))
    end   = dt.datetime.now()
    #print('Seconds to Compute . . . . . . . . . . . . . . :' + (end - start).seconds)
    print('. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ')
    if i%111 == 0:
        df0.to_csv('c:\\thomas\\ib\\'+file+'.csv')  
df0.to_csv('c:\\thomas\\ib\\'+file+'.csv')  

update dfx and append to df0
Server Version: 76
TWS Time at connection:20161026 21:34:45 BRT
............................................................................... 
2016-10-26 21:44:54+00:00
Iteration. . . . . . . . . . . . . . . . . . . :1
Records. . . . . . . . . . . . . . . . . . . . :6110
Trades. . . . . . . . . . . . . . . . . . . . .:3017
Last Trade Type. . . . . . . . . . . . . . . . :B4TLL
Minutes. . . . . . . . . . . . . . . . . . . . :1304
Time of Trade. . . . . . . . . . . . . . . . . :1304
Reference Price. . . . . . . . . . . . . . . . :1.0911
Price. . . . . . . . . . . . . . . . . . . . . :1.09107
Trigger. . . . . . . . . . . . . . . . . . . . :0.00020
Limit of Control . . . . . . . . . . . . . . . :0.00017
Cumulative Profit/Loss. . . . . .  . . . . . . :-507
Cumulative Commissions . . . . . . . . . . . . :1114.0
Wins in Streak . . . . . . . . . . . . . . . . :0
Streak Number. . . . . . . . . . . . . . . . . :1
Reversal Count . . . . . . . . . . . . . . . . :1
Rep